In [109]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from nltk.classify import ClassifierI
from statistics import mode

In [110]:
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.metrics import roc_curve, auc

In [111]:
# 7 Classifiers combined to one (SVC classifier is not included)
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
        
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [112]:
documents = [(list(movie_reviews.words(fileid)), category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]

# Disabled randomizing to be able to check against positive or negative data.
random.shuffle(documents)

In [113]:
all_words = []
for w in movie_reviews.words():
    all_words.append(w.lower())
    
all_words = nltk.FreqDist(all_words)
# print(all_words.most_common(15))
# print(all_words["movie"])

In [114]:
word_features = list(all_words.keys())

In [115]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [116]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [117]:
# # SLMRD/test/ contains neg/*.txt and pos/*.txt files
# # SLMRD test set
# import os
# PathNeg = "SLMRD/test/neg/"
# PathPos = "SLMRD/test/pos/"
# filelistNeg = os.listdir(PathNeg)
# filelistPos = os.listdir(PathPos)
# slmrd_negative_reviews = []
# slmrd_positive_reviews = []
# slmrd_test_reviews = []

# for i in filelistNeg:
#     if i.endswith(".txt"):
#         with open(PathNeg + i, 'r', encoding="utf-8") as f:
#             for line in f:
#                 slmrd_negative_reviews.append((line, 'neg'))
#         f.close()


# for i in filelistPos:
#     if i.endswith(".txt"):
#         with open(PathPos + i, 'r', encoding="utf-8") as f:
#             for line in f:
#                 slmrd_positive_reviews.append((line, 'pos'))
#         f.close()


# # print('Negative: ', slmrd_negative_reviews[1])
# # print('Positive: ', slmrd_positive_reviews[1])
# for (words, sentiment) in slmrd_positive_reviews + slmrd_negative_reviews:
#     words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
#     slmrd_test_reviews.append((words_filtered, sentiment))
    
# slmrd_featuresets = [(find_features(rev), category) for (rev, category) in slmrd_test_reviews]

# # print('SLMRD Featureset: ', slmrd_featuresets[1])

In [118]:
# testing_set = slmrd_featuresets
# training_set = featuresets

training_set = featuresets[:1900]
testing_set = featuresets[1900:]

# Positive data example:
# training_set = featuresets[:1900]
# testing_set = featuresets[1900:]

# Negative data example:
# training_set = featuresets[100:]
# testing_set = featuresets[:100]

In [119]:
# NB_classifier = nltk.NaiveBayesClassifier()

## Load an existing classifier.pickle file to the classifier, instead of re-training it every time.
# NB_classifier_f = open("SavedTrainingData/NB_classifier.pickle", "rb")
# NB_classifier = pickle.load(NB_classifier_f)
# NB_classifier_f.close()

## Uncomment below to re-train the training set.
NB_classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Original Naive Bayes Classifier accuracy percent: ", (nltk.classify.accuracy(NB_classifier, testing_set))*100)
# classifier.show_most_informative_features(15)

## Save the classifier as a .pickle file. Uncomment below to save the trained data.
# print("Saving training data...")
# save_NB_classifier = open("SavedTrainingData/NB_classifier.pickle", "wb")
# pickle.dump(NB_classifier, save_NB_classifier)
# save_NB_classifier.close()

Original Naive Bayes Classifier accuracy percent:  80.0


In [120]:
MNB_classifier = SklearnClassifier(MultinomialNB())

## Load an existing MNB_classifier.pickle file to the classifier, instead of re-training it every time.
# MNB_classifier_f = open("SavedTrainingData/MNB_classifier.pickle", "rb")
# MNB_classifier = pickle.load(MNB_classifier_f)
# MNB_classifier_f.close()

## Uncomment below to re-train the training set.
MNB_classifier.train(training_set)

print("MNB_classifier accuracy percent: ", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

## Save the classifier as a .pickle file. Uncomment below to save the trained data.
# print("Saving training data...")
# save_MNB_classifier = open("SavedTrainingData/MNB_classifier.pickle", "wb")
# pickle.dump(MNB_classifier, save_MNB_classifier)
# save_MNB_classifier.close()

MNB_classifier accuracy percent:  82.0


In [121]:
BNB_classifier = SklearnClassifier(BernoulliNB())

## Load an existing BNB_classifier.pickle file to the classifier, instead of re-training it every time.
# BNB_classifier_f = open("SavedTrainingData/BNB_classifier.pickle", "rb")
# BNB_classifier = pickle.load(BNB_classifier_f)
# BNB_classifier_f.close()

## Uncomment below to re-train the training set.
BNB_classifier.train(training_set)

print("BernoulliNB_classifier accuracy percent:",(nltk.classify.accuracy(BNB_classifier, testing_set))*100)

## Save the classifier as a .pickle file. Uncomment below to save the trained data.
# print("Saving training data...")
# save_BNB_classifier = open("SavedTrainingData/BNB_classifier.pickle", "wb")
# pickle.dump(BNB_classifier, save_BNB_classifier)
# save_BNB_classifier.close()

BernoulliNB_classifier accuracy percent: 80.0


In [122]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())

## Load an existing LogisticRegression_classifier.pickle file to the classifier, instead of re-training it every time.
# LogisticRegression_classifier_f = open("SavedTrainingData/LogisticRegression_classifier.pickle", "rb")
# LogisticRegression_classifier = pickle.load(LogisticRegression_classifier_f)
# LogisticRegression_classifier_f.close()

## Uncomment below to re-train the training set.
LogisticRegression_classifier.train(training_set)

print("LogisticRegression_classifier accuracy percent:", 
      (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

## Save the classifier as a .pickle file. Uncomment below to save the trained data.
# print("Saving training data...")
# save_LogisticRegression_classifier = open("SavedTrainingData/LogisticRegression_classifier.pickle", "wb")
# pickle.dump(LogisticRegression_classifier, save_LogisticRegression_classifier)
# save_LogisticRegression_classifier.close()

LogisticRegression_classifier accuracy percent: 85.0


In [123]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())

## Load an existing SGDClassifier_classifier.pickle file to the classifier, instead of re-training it every time.
# SGDClassifier_classifier_f = open("SavedTrainingData/SGDClassifier_classifier.pickle", "rb")
# SGDClassifier_classifier = pickle.load(SGDClassifier_classifier_f)
# SGDClassifier_classifier_f.close()

## Uncomment below to re-train the training set.
SGDClassifier_classifier.train(training_set)

print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

## Save the classifier as a .pickle file. Uncomment below to save the trained data.
# print("Saving training data...")
# save_SGDClassifier_classifier = open("SavedTrainingData/SGDClassifier_classifier.pickle", "wb")
# pickle.dump(SGDClassifier_classifier, save_SGDClassifier_classifier)
# save_SGDClassifier_classifier.close()

D:\Applications\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 87.0


In [124]:
# SVC_classifier = SklearnClassifier(SVC())

# ## Load an existing SVC_classifier.pickle file to the classifier, instead of re-training it every time.
# SVC_classifier_f = open("SavedTrainingData/SVC_classifier.pickle", "rb")
# SVC_classifier = pickle.load(SVC_classifier_f)
# SVC_classifier_f.close()

# ## Uncomment below to re-train the training set.
# # SVC_classifier.train(training_set)

# print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

# ## Save the classifier as a .pickle file. Uncomment below to save the trained data.
# # print("Saving training data...")
# # save_SVC_classifier = open("SavedTrainingData/SVC_classifier.pickle", "wb")
# # pickle.dump(SVC_classifier, save_SVC_classifier)
# # save_SVC_classifier.close()

In [125]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())

## Load an existing LinearSVC_classifier.pickle file to the classifier, instead of re-training it every time.
# LinearSVC_classifier_f = open("SavedTrainingData/LinearSVC_classifier.pickle", "rb")
# LinearSVC_classifier = pickle.load(LinearSVC_classifier_f)
# LinearSVC_classifier_f.close()

## Uncomment below to re-train the training set.
LinearSVC_classifier.train(training_set)

print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

## Save the classifier as a .pickle file. Uncomment below to save the trained data.
# print("Saving training data...")
# save_LinearSVC_classifier = open("SavedTrainingData/LinearSVC_classifier.pickle", "wb")
# pickle.dump(LinearSVC_classifier, save_LinearSVC_classifier)
# save_LinearSVC_classifier.close()

LinearSVC_classifier accuracy percent: 84.0


In [126]:
NuSVC_classifier = SklearnClassifier(NuSVC())

## Load an existing NuSVC_classifier.pickle file to the classifier, instead of re-training it every time.
# NuSVC_classifier_f = open("SavedTrainingData/NuSVC_classifier.pickle", "rb")
# NuSVC_classifier = pickle.load(NuSVC_classifier_f)
# NuSVC_classifier_f.close()

## Uncomment below to re-train the training set.
NuSVC_classifier.train(training_set)

print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

## Save the classifier as a .pickle file. Uncomment below to save the trained data.
# print("Saving training data...")
# save_NuSVC_classifier = open("SavedTrainingData/NuSVC_classifier.pickle", "wb")
# pickle.dump(NuSVC_classifier, save_NuSVC_classifier)
# save_NuSVC_classifier.close()

NuSVC_classifier accuracy percent: 85.0


In [128]:
voted_classifiers = VoteClassifier(NB_classifier, 
                                   MNB_classifier, 
                                   BNB_classifier, 
                                   LogisticRegression_classifier, 
                                   SGDClassifier_classifier, 
                                   LinearSVC_classifier, 
                                   NuSVC_classifier)
print("voted_classifiers accuracy percent:", (nltk.classify.accuracy(voted_classifiers, testing_set))*100)

print("Classification:", voted_classifiers.classify(testing_set[0][0]), "Confidence %:", voted_classifiers.confidence(testing_set[0][0])*100)
print("Classification:", voted_classifiers.classify(testing_set[5][0]), "Confidence %:", voted_classifiers.confidence(testing_set[5][0])*100)
print("Classification:", voted_classifiers.classify(testing_set[17][0]), "Confidence %:", voted_classifiers.confidence(testing_set[17][0])*100)
print("Classification:", voted_classifiers.classify(testing_set[55][0]), "Confidence %:", voted_classifiers.confidence(testing_set[55][0])*100)
print("Classification:", voted_classifiers.classify(testing_set[281][0]), "Confidence %:", voted_classifiers.confidence(testing_set[281][0])*100)
print("Classification:", voted_classifiers.classify(testing_set[1259][0]), "Confidence %:", voted_classifiers.confidence(testing_set[1259][0])*100)



voted_classifiers accuracy percent: 88.0
Classification: neg Confidence %: 85.71428571428571
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 57.14285714285714


IndexError: list index out of range

In [129]:
# print(testing_set[0][0]," ",
# testing_set[5][0]," ",
# testing_set[17][0]," ",
# testing_set[55][0]," ",
# testing_set[281][0]," ",
# testing_set[1259][0])